In [3]:
with open("sources/the_rubaiyat.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

cleaned = []
prev_blank = False
for line in lines:
    line = line.strip()
    if not line:
        if not prev_blank:
            cleaned.append("")  # keep only one blank line
        prev_blank = True
    else:
        cleaned.append(line)
        prev_blank = False

with open("the_rubaiyat_clean.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(cleaned))

print("Cleaning done. Saved as the_rubaiyat_clean.txt")


Cleaning done. Saved as the_rubaiyat_clean.txt
